In [1]:
import pandas as pd
pd.options.mode.copy_on_write = True
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
import os

import itertools

import COMBINE_harmonizer
from COMBINE_harmonizer import cfg

## 01. init

In [2]:
root_dir = '../'

In [3]:
COMBINE_harmonizer.init(f'{root_dir}/config.yaml')

## 02. Dictionary

In [4]:
data_dict_filename = f'{root_dir}/Dictionary_HIE_clinical_variables.xlsx'

In [5]:
df_dict_main = COMBINE_harmonizer.load_data_dict(data_dict_filename, sheet_name=COMBINE_harmonizer.SHEET_MAIN)
df_dict_followup = COMBINE_harmonizer.load_data_dict(data_dict_filename, sheet_name=COMBINE_harmonizer.SHEET_FOLLOW_UP)
df_dict_derived_data = COMBINE_harmonizer.load_data_dict(data_dict_filename, sheet_name=COMBINE_harmonizer.SHEET_DERIVED_DATA)

## 03. Mapping

In [6]:
_MAIN_ORDER_MAP = {row[COMBINE_harmonizer.DATA_DICT_VAR_NAME]: idx for idx, row in df_dict_main.iterrows() if pd.isnull(row[COMBINE_harmonizer.DATA_DICT_VAR_NAME]) == False}
_FOLLOWUP_ORDER_MAP = {row[COMBINE_harmonizer.DATA_DICT_VAR_NAME]: idx for idx, row in df_dict_followup.iterrows() if pd.isnull(row[COMBINE_harmonizer.DATA_DICT_VAR_NAME]) == False}
_ANALYSIS_ORDER_MAP = {row[COMBINE_harmonizer.DATA_DICT_VAR_NAME]: idx for idx, row in df_dict_derived_data.iterrows() if pd.isnull(row[COMBINE_harmonizer.DATA_DICT_VAR_NAME]) == False}

# XXX hack for duplicated columns
_MAIN_ORDER_MAP['MRIDate'] = 612
_MAIN_ORDER_MAP['MRITime'] = 613
_MAIN_ORDER_MAP['birthDate'] = 131

In [7]:
_MAIN_ORDER_MAP

{'center': 688,
 'subjectID': 690,
 'siteID': 2,
 'birthDate': 131,
 'birthNumber': 4,
 'screenComment': 5,
 'coreTempLess32p5COverEq2Hr_e': 6,
 'coreTempLess33p5COver1Hr_e': 7,
 'coreTempLess34COver1Hr_e': 8,
 'first6HrCoolByClinicalProtocol_e': 9,
 'chromosomalAbnormality_e': 10,
 'majorCongenitalAnomaly_e': 11,
 'birthWeightLessEq1800g_e': 12,
 'infantUnlikelySurvive_e': 13,
 'first60MinAllBloodGasPHGreater7p15BaseDeficitLess10mEqPerL_e': 14,
 'postnatalAgeLess6HrOrGreater24Hr_e': 15,
 'enrolledConflictingTrial_e': 16,
 'first60MinAnyBloodGasPHLessEq7_i': 17,
 'first60MinAnyBloodGasBaseDeficitGreaterEq16mEqPerL_i': 18,
 'historyPerinatalEvent_i': 19,
 'at10MinApgarLessEq5OrVent_i': 20,
 'randomEligible': 21,
 'consentStatus': 22,
 'noConsentReason': 23,
 'noInStudyReason': 24,
 'random': 25,
 'noRandomReason': 26,
 'noRandomReasonText': 27,
 'randomDate': 28,
 'randomTime': 29,
 'randomNumber': 30,
 'randomTreatmentAssign': 31,
 'randomTreatmentReceive': 32,
 'treatmentBlanketType':

In [8]:
_FOLLOWUP_ORDER_MAP

{'followupCenter': 979,
 'siteID': 980,
 'birthDate': 981,
 'visitDate': 982,
 'birthNumber': 983,
 'center': 984,
 'subjectID': 985,
 'followupID': 986,
 'center_orig': 987,
 'SESVisitDate': 988,
 'SESBirthDate': 989,
 'chronologicalAge_mo': 990,
 'correctedAge_mo': 991,
 'underStateSupervision': 992,
 'primaryCaretaker': 993,
 'otherCaretaker': 994,
 'maritalStatusPrimaryCaretaker': 995,
 'livingArrangementChild': 996,
 'numberPeopleInChildHousehold': 997,
 'otherContributeMoneyToChildHousehold': 998,
 'educationPrimaryCaretaker': 999,
 'educationOtherCaretaker': 1000,
 'workPrimaryCaretaker': 1001,
 'workOtherCaretaker': 1002,
 'inSchoolPrimaryCaretaker': 1003,
 'inSchoolOtherCaretaker': 1004,
 'totalIncomeChildHousehold': 1005,
 'medicalInsuranceChild': 1006,
 'primaryLanguageChild': 1007,
 'primaryLanguageChildOtherText': 1008,
 'isSecondaryLanguageChild': 1009,
 'secondaryLanguageChild': 1010,
 'secondaryLanguageChildOtherText': 1011,
 'numberPlaceChildLive': 1012,
 'zipcode': 10

In [9]:
_ANALYSIS_ORDER_MAP

{'center': 1334,
 'followupCenter': 1335,
 'subjectID': 1336,
 'followupID': 1337,
 'acidosis': 1338,
 'ageDeath_day': 1339,
 'ageRand_hr': 1340,
 'baselineAnticonvulsants': 1341,
 'dischargeAnticonvulsants': 1342,
 'inotropicAgent': 1343,
 'perinatalSentinelEvent': 1344,
 'dischargeSeizure': 1345,
 'Apgar10minLt5': 1346,
 'Apgar10minLte5': 1347,
 'Apgar5minLte5': 1348,
 'bloodGasBaseDeficit_mEqPerL': 1349,
 'bloodGasPH': 1350,
 'emergencyCSection': 1351,
 'encephalopathyLevel': 1352,
 'inotropicAgentBaseline': 1353,
 'maleSex': 1354,
 'maternalEducation': 1355,
 'motherInsurancePublic': 1356,
 'motherRace': 1357,
 'treatmentAssignmentDuration_hr': 1358,
 'treatmentAssignmentTemperature': 1359,
 'bloodGasBaseDeficit_mEqPerLSrc': 1360,
 'bloodGasPHSrc': 1361,
 'usualCoolingTreatmentGroup': 1362,
 'blindness': 1363,
 'moderateSevereCerebralPalsy': 1364,
 'cerebralPalsy': 1365,
 'gastrostomyTube': 1366,
 'grossMotorFunctionLevel': 1367,
 'hearingImpairedWithAid': 1368,
 'hearingImpairedLe

## 04. Filenames and Merge

In [10]:
out_dir = cfg.config['out_dir']
_DIRS = {
    'LH': f'{out_dir}/out-LH-normalized',
    'OC': f'{out_dir}/out-OC-normalized',
}

_FILENAMES = [each['name'] for each in COMBINE_harmonizer.FILENAME_INFOS]

_FILENAME_INFO_MAP = {each['name']: each for each in COMBINE_harmonizer.FILENAME_INFOS}

_OUT_DIR = f'{out_dir}/out-merged-normalized'
os.makedirs(_OUT_DIR, exist_ok=True)

In [11]:
def _get_order(x, order_map):
    if x not in order_map:
        print(f'[WARN] not in order_map: {x}')
        return COMBINE_harmonizer.MAX_INT

    return order_map[x]


def _column_and_orig(column, columns_with_orig):
    ret = [column]

    # XXX remove .orig in merge
    # if column + '.orig' in columns_with_orig:
    #     ret += [column + '.orig']

    return ret


def _reorder_columns(df, filename):
    columns_without_orig = list(filter(lambda x: not x.endswith('.orig'), df.columns))
    columns_with_orig = list(filter(lambda x: x.endswith('.orig'), df.columns))

    each_filename_info = _FILENAME_INFO_MAP[filename]
    order_map = _MAIN_ORDER_MAP
    if each_filename_info['data_dict'] == COMBINE_harmonizer.SHEET_FOLLOW_UP:
        order_map = _FOLLOWUP_ORDER_MAP
    elif each_filename_info['data_dict'] == COMBINE_harmonizer.SHEET_DERIVED_DATA:
        order_map = _ANALYSIS_ORDER_MAP

    columns_without_orig.sort(key=lambda x: _get_order(x, order_map))
    columns_with_orig.sort(key=lambda x: _get_order(x[:-5], order_map))

    reserved_columns = list(filter(lambda x: x in columns_without_orig, COMBINE_harmonizer.RESERVED_COLUMNS))
    other_columns = list(filter(lambda x: x not in COMBINE_harmonizer.RESERVED_COLUMNS, columns_without_orig))

    columns_and_origs = list(itertools.chain.from_iterable([_column_and_orig(each, columns_with_orig) for each in other_columns]))

    columns = reserved_columns + columns_and_origs

    return df[columns]

In [12]:
for idx, filename in enumerate(_FILENAMES):
    df = None
    for study, each_dir in _DIRS.items():
        each_filename = os.sep.join([each_dir, filename])
        if not os.path.exists(each_filename):
            continue
        each_df = pd.read_csv(each_filename, dtype='O')
        each_df['_study'] = study

        if df is None:
            df = each_df
        else:
            df = pd.concat([df, each_df])

    if df is None:
        print(f'[WARN] ({idx}/{len(_FILENAMES)}) not df: filename: {filename}')
        continue

    df = _reorder_columns(df, filename)

    out_filename = os.sep.join([_OUT_DIR, filename])

    df.to_csv(out_filename, index=False)

    print(f'[INFO] ({idx}/{len(_FILENAMES)}) done: filename: {filename} out_filename: {out_filename}')

[WARN] not in order_map: uniqueID
[WARN] not in order_map: _study
[WARN] not in order_map: uniqueID
[INFO] (0/72) done: filename: 00-02-screening.csv out_filename: /Volumes/neuro/labs/grantlab/research/chuanheng.hsiao/HIE-out/out-merged-normalized/00-02-screening.csv


[WARN] not in order_map: uniqueID
[WARN] not in order_map: _study
[WARN] not in order_map: uniqueID
[INFO] (1/72) done: filename: 00-12-neuro-exam.csv out_filename: /Volumes/neuro/labs/grantlab/research/chuanheng.hsiao/HIE-out/out-merged-normalized/00-12-neuro-exam.csv
[WARN] not in order_map: uniqueID
[WARN] not in order_map: _study
[WARN] not in order_map: uniqueID


[INFO] (2/72) done: filename: 01-02-screening.csv out_filename: /Volumes/neuro/labs/grantlab/research/chuanheng.hsiao/HIE-out/out-merged-normalized/01-02-screening.csv
[WARN] not in order_map: uniqueID
[WARN] not in order_map: _study
[WARN] not in order_map: uniqueID
[INFO] (3/72) done: filename: 01-03-maternal-demographics.csv out_filename: /Volumes/neuro/labs/grantlab/research/chuanheng.hsiao/HIE-out/out-merged-normalized/01-03-maternal-demographics.csv


[WARN] not in order_map: uniqueID
[WARN] not in order_map: _study
[WARN] not in order_map: uniqueID
[INFO] (4/72) done: filename: 01-04-pregnancy-history.csv out_filename: /Volumes/neuro/labs/grantlab/research/chuanheng.hsiao/HIE-out/out-merged-normalized/01-04-pregnancy-history.csv
[WARN] not in order_map: uniqueID
[WARN] not in order_map: _study
[WARN] not in order_map: uniqueID


[INFO] (5/72) done: filename: 01-05-labor-delivery.csv out_filename: /Volumes/neuro/labs/grantlab/research/chuanheng.hsiao/HIE-out/out-merged-normalized/01-05-labor-delivery.csv
[WARN] not in order_map: uniqueID
[WARN] not in order_map: _study
[WARN] not in order_map: uniqueID


...

[WARN] not in order_map: uniqueID
[WARN] not in order_map: uniqueFollowupID
[WARN] not in order_map: _study
[WARN] not in order_map: uniqueID
[INFO] (59/72) done: filename: 20-05-gmfcs.csv out_filename: /Volumes/neuro/labs/grantlab/research/chuanheng.hsiao/HIE-out/out-merged-normalized/20-05-gmfcs.csv
[WARN] not in order_map: uniqueID
[WARN] not in order_map: uniqueFollowupID
[WARN] not in order_map: _study
[WARN] not in order_map: uniqueID


[INFO] (60/72) done: filename: 20-06-status.csv out_filename: /Volumes/neuro/labs/grantlab/research/chuanheng.hsiao/HIE-out/out-merged-normalized/20-06-status.csv
[WARN] not in order_map: uniqueID
[WARN] not in order_map: uniqueFollowupID
[WARN] not in order_map: _flatten_index
[WARN] not in order_map: _study
[WARN] not in order_map: uniqueID
[INFO] (61/72) done: filename: 20-07-readmission.csv out_filename: /Volumes/neuro/labs/grantlab/research/chuanheng.hsiao/HIE-out/out-merged-normalized/20-07-readmission.csv


[WARN] not in order_map: uniqueID
[WARN] not in order_map: uniqueFollowupID
[WARN] not in order_map: _study
[WARN] not in order_map: uniqueID
[INFO] (62/72) done: filename: 20-08-lost.csv out_filename: /Volumes/neuro/labs/grantlab/research/chuanheng.hsiao/HIE-out/out-merged-normalized/20-08-lost.csv
[WARN] not in order_map: uniqueID
[WARN] not in order_map: _study
[WARN] not in order_map: uniqueID


[INFO] (63/72) done: filename: 30-01-secondary.csv out_filename: /Volumes/neuro/labs/grantlab/research/chuanheng.hsiao/HIE-out/out-merged-normalized/30-01-secondary.csv
[WARN] not in order_map: uniqueID
[WARN] not in order_map: _study
[WARN] not in order_map: uniqueID
[INFO] (64/72) done: filename: 30-02-outcome.csv out_filename: /Volumes/neuro/labs/grantlab/research/chuanheng.hsiao/HIE-out/out-merged-normalized/30-02-outcome.csv


[WARN] not in order_map: uniqueID
[WARN] not in order_map: _study
[WARN] not in order_map: uniqueID
[INFO] (65/72) done: filename: 30-03-mri.csv out_filename: /Volumes/neuro/labs/grantlab/research/chuanheng.hsiao/HIE-out/out-merged-normalized/30-03-mri.csv
[WARN] (66/72) not df: filename: 31-02-total-modified-sarnat.csv
[WARN] (67/72) not df: filename: 31-03-mri.csv
[WARN] (68/72) not df: filename: 31-04-pse.csv
[WARN] (69/72) not df: filename: 31-05-disability-level-death.csv
[WARN] (70/72) not df: filename: 31-06-emergency-csection.csv
[WARN] (71/72) not df: filename: 31-07-length-of-stay.csv
